In [1]:
from scipy.spatial.transform import Rotation
import numpy as np
import json

# Rotations In USGSCSM Sensor Models
In USGSCSM, we primarily use quaternions and ocassionally use Euler angles.

## Quaternions
USGSCSM sensor models always store the rotation from the sensor reference frame to the target reference frame as quaternions. The framing sensor model stores a single quaternion and the line scan sensor model stores a sequence of quaternions that are equally spaced across the duration of the image. Similar to ISIS sensor models, the line scan sensor model interpolates to get the rotation at any given time.

## Euler Angles
The one place that the USGSCSM uses Euler angles is in pointing correction for the line scan sensor model. The line scan sensor model retains the original rotation quaternions and adds an additional bias rotation defined by a second degree polynomial for each Euler angle. In ISIS this is equivalent to the overexisting parameter in jigsaw.

## An Example
Let's look at an MDIS NAC ISD

In [2]:
with open('data/EN0213110924M.json') as fh:
    isd = json.load(fh)
print(isd)

{'center_ephemeris_time': 357895527.7312159, 'detector_center': {'line': 512.5, 'sample': 512.5}, 'detector_line_summing': 2, 'detector_sample_summing': 2, 'fikid': -236820, 'file': '/usgs/shareall/thare/MDIS_StereoPair_maybe/EN0213110924M.IMG', 'filter_number': 'N/A', 'focal2pixel_lines': [0.0, 0.0, 71.42857143], 'focal2pixel_samples': [0.0, 71.42857143, 0.0], 'focal_length_model': {'focal_length': 549.2191106302377}, 'ikid': -236820, 'image_lines': 512, 'image_samples': 512, 'instrument_id': 'MSGR_MDIS_NAC', 'interpolation_method': 'lagrange', 'line_exposure_duration': 0.019, 'metakernel': '/Users/jmapel/ale/kernels/mess/msgr_2011_v10.tm', 'name_model': 'USGS_ASTRO_FRAME_SENSOR_MODEL', 'name_platform': 'Generic Platform', 'name_sensor': 'Generic Sensor', 'number_of_ephemerides': 1, 'number_of_quaternions': 1, 'radii': {'semimajor': 2439.4, 'semiminor': 2439.4, 'unit': 'km'}, 'reference_frame': 'IAU_MERCURY', 'reference_height': {'minheight': 0, 'maxheight': 1000, 'unit': 'm'}, 'senso

The rotation from the sensor reference frame to the target reference frame is stored in the `sensor_orientation` as `quaternions`. In order for the line scan sensor model and the frame sensor model, the quaternions are stored as a 2d list regardless of how many there are.

<span style="color:blue">Using the information from the MDIS-NAC ISD, rotate the look vector (7.168, 7.168, 550) from the sensor reference frame to the target reference frame</span>

In [3]:
# Note that USGSCSM orders its quaternions the same as scipy, xi + yj + zk + w
look_vec = np.array([7.168, 7.168, 550])

sensor_to_target = Rotation.from_quat(isd['sensor_orientation']['quaternions'])

print('The rotated look vector is', sensor_to_target.apply(look_vec))

The rotated look vector is [[ 304.6492836  -361.53955175 -281.21295661]]


<span style="color:blue">Now add an additional pointing correction of a 2.5 degree rotation about the X-axis, a 1.7 degree rotation about the Y-axis and a -3 degree rotation about the Z-axis applied before the original quaternion. What does the look vector (7.168, 7.168, 550) rotate to in the target reference frame?</span>

In [5]:
correction_rot = Rotation.from_euler('xyz', [2.5, 1.7, -3], degrees=True)

adjusted_sensor_to_target = sensor_to_target * correction_rot

print('The rotated look vector is', adjusted_sensor_to_target.apply(look_vec))

The rotated look vector is [[ 301.2100811  -344.59883321 -305.16698977]]
